In [11]:
import psycopg2
import geopandas as gpd
import folium
import fiona

In [29]:
rappi = "ST_Buffer(ST_GeomFromText('POINT(981644.32 5979177.89)', 3857), 1000)"

query = """
WITH line_polygons AS (
WITH multi(way) AS (
  SELECT ST_Node(ST_Collect(way)) FROM planet_osm_line
  WHERE ST_Intersects(way, {area})
)
SELECT (ST_Dump(ST_Polygonize(way))).geom AS geometry FROM multi
),
aoi_hulls AS (
  SELECT hull FROM aois WHERE ST_Intersects(hull, {area})
),
covered_polygons AS(
  SELECT way FROM planet_osm_polygon, aoi_hulls WHERE ST_Covers(aoi_hulls.hull, way)
)
SELECT geometry FROM line_polygons, aoi_hulls WHERE ST_Area(ST_Intersection(aoi_hulls.hull, geometry)) / ST_Area(geometry) > 0.8
UNION ALL
SELECT way FROM covered_polygons
""".format(area=rappi)

with psycopg2.connect("") as conn:
    street_polygons = gpd.read_postgis(query, conn, geom_col='geometry')
    
street_polygons.crs = fiona.crs.from_epsg(3857)

m = folium.Map(location=[47.226, 8.818], zoom_start=17)
folium.GeoJson(street_polygons).add_to(m)
m